In [1]:
from pyAudioAnalysis import audioTrainTest as aT
from pyAudioAnalysis import MidTermFeatures as aF
from pyAudioAnalysis import audioAnalysis as aA
from pathlib import Path
import numpy as np
import tensorflow as tf
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

You pressed Ctrl+C! - EXIT


SystemExit: 0

In [2]:
directory_in_str = "emotionData/Individual"
pathlist = Path(directory_in_str).glob('**/*.wav')

listOfDirs = []
for path in pathlist:
    path_in_str = str(path)
    parent = path.parents[0]
    if str(parent) not in listOfDirs:
        listOfDirs.append(str(parent))

mtWin = 1.0
mtStep = 1.0
stWin = aT.shortTermWindow
stStep = aT.shortTermStep
classifierType = "svm"
modelName = "models/svm_1v1_1.0.0"
beat = False

In [3]:
features, class_names, file_names, feature_names = aF.multiple_directory_feature_extraction(listOfDirs, mtWin, mtStep, stWin, stStep, beat)

ation\Voice_Mary_Admiration_46.wav
Analyzing file 43 of 96: emotionData\Individual\Admiration\Voice_Mary_Admiration_47.wav
Analyzing file 44 of 96: emotionData\Individual\Admiration\Voice_Mary_Admiration_48.wav
Analyzing file 45 of 96: emotionData\Individual\Admiration\Voice_Mary_Admiration_49.wav
Analyzing file 46 of 96: emotionData\Individual\Admiration\Voice_Mary_Admiration_5.wav
Analyzing file 47 of 96: emotionData\Individual\Admiration\Voice_Mary_Admiration_50.wav
Analyzing file 48 of 96: emotionData\Individual\Admiration\Voice_Mary_Admiration_51.wav
Analyzing file 49 of 96: emotionData\Individual\Admiration\Voice_Mary_Admiration_52.wav
Analyzing file 50 of 96: emotionData\Individual\Admiration\Voice_Mary_Admiration_53.wav
Analyzing file 51 of 96: emotionData\Individual\Admiration\Voice_Mary_Admiration_54.wav
Analyzing file 52 of 96: emotionData\Individual\Admiration\Voice_Mary_Admiration_55.wav
Analyzing file 53 of 96: emotionData\Individual\Admiration\Voice_Mary_Admiration_56.wa

In [4]:
print(np.array(features).shape)
print(features[0].shape)
print(class_names)
print()
print(feature_names[0])
print(feature_names[0].index('zcr_std'))
print(len(feature_names))

(20,)
(96, 136)
['Admiration', 'Amusement', 'Anger', 'Compassion', 'Contempt', 'Contentment', 'Disappointment', 'Disgust', 'Fear', 'Guilt', 'Hate', 'Interest', 'Joy', 'Love', 'Pleasure', 'Pride', 'Regret', 'Relief', 'Sadness', 'Shame']

['zcr_mean', 'energy_mean', 'energy_entropy_mean', 'spectral_centroid_mean', 'spectral_spread_mean', 'spectral_entropy_mean', 'spectral_flux_mean', 'spectral_rolloff_mean', 'mfcc_1_mean', 'mfcc_2_mean', 'mfcc_3_mean', 'mfcc_4_mean', 'mfcc_5_mean', 'mfcc_6_mean', 'mfcc_7_mean', 'mfcc_8_mean', 'mfcc_9_mean', 'mfcc_10_mean', 'mfcc_11_mean', 'mfcc_12_mean', 'mfcc_13_mean', 'chroma_1_mean', 'chroma_2_mean', 'chroma_3_mean', 'chroma_4_mean', 'chroma_5_mean', 'chroma_6_mean', 'chroma_7_mean', 'chroma_8_mean', 'chroma_9_mean', 'chroma_10_mean', 'chroma_11_mean', 'chroma_12_mean', 'chroma_std_mean', 'delta zcr_mean', 'delta energy_mean', 'delta energy_entropy_mean', 'delta spectral_centroid_mean', 'delta spectral_spread_mean', 'delta spectral_entropy_mean', 'del

In [5]:
def extract_features_and_train(features, classifier_type, model_name, train_percentage=0.90):

    n_feats = features[0].shape[1]
    feature_names = ["features" + str(d + 1) for d in range(n_feats)]

    # STEP B: classifier Evaluation and Parameter Selection:
    if classifier_type == "svm" or classifier_type == "svm_rbf":
        classifier_par = np.array([0.001, 0.01,  0.5, 1.0, 5.0, 10.0, 20.0])
    elif classifier_type == "randomforest":
        classifier_par = np.array([10, 25, 50, 100, 200, 500])
    elif classifier_type == "knn":
        classifier_par = np.array([1, 3, 5, 7, 9, 11, 13, 15])        
    elif classifier_type == "gradientboosting":
        classifier_par = np.array([10, 25, 50, 100, 200, 500])
    elif classifier_type == "extratrees":
        classifier_par = np.array([10, 25, 50, 100, 200, 500])

    # get optimal classifeir parameter:
    temp_features = []
    for feat in features:
        temp = []
        for i in range(feat.shape[0]):
            temp_fv = feat[i, :]
            if (not np.isnan(temp_fv).any()) and (not np.isinf(temp_fv).any()):
                temp.append(temp_fv.tolist())
            else:
                print("NaN Found! Feature vector not used for training")
        temp_features.append(np.array(temp))
    features = temp_features

    best_param, best_score = aT.evaluate_classifier(features, class_names, 100, classifier_type,
                                     classifier_par, 1, train_percentage)

    print("Selected params: {0:.5f}".format(best_param))

    return best_score

In [20]:
features_copy = [features[i] for i in range(len(features))]
feature_names_copy = feature_names[0]

perm_features = []
accuracies = []
perm_feature_names = []

print(features_copy[0].shape)
print(feature_names_copy)

while feature_names_copy:
    temp_accs = []
    for i in range(len(feature_names_copy)):
        print("\nEvaluating {} | {} / {}".format(feature_names_copy[i], i, len(feature_names_copy)))
        if perm_features:
            train_features = [np.concatenate((perm_features[j], features_copy[j][:,i,np.newaxis]), axis=1) for j in range(len(features_copy))]
        else:   
            train_features = [features_copy[j][:,i,np.newaxis] for j in range(len(features_copy))]
        print(train_features[0].shape)
        temp_accs.append(extract_features_and_train(train_features, "svm", "models/tempSvm_1v1_1.0.0"))
    
    best_acc = max(temp_accs)
    best_index = temp_accs.index(best_acc)
    accuracies.append(best_acc)
    if perm_features:
        perm_features = [np.concatenate((perm_features[j], features_copy[j][:,best_index,np.newaxis]), axis=1) for j in range(len(features_copy))]
    else:
        perm_features = [features_copy[j][:,best_index,np.newaxis] for j in range(len(features_copy))]
    features_copy = [np.delete(features[i], best_index, axis=1) for i in range(len(features_copy))]
    added_feature = feature_names_copy.pop(best_index)
    perm_feature_names.append(added_feature)
    print("Adding {} to the perm features".format(added_feature))
    print("New accuracy is {}".format(best_acc))
    print("New Perm Features list is {}".format(perm_feature_names))


(164, 136)
['zcr_mean', 'energy_mean', 'energy_entropy_mean', 'spectral_centroid_mean', 'spectral_spread_mean', 'spectral_entropy_mean', 'spectral_flux_mean', 'spectral_rolloff_mean', 'mfcc_1_mean', 'mfcc_2_mean', 'mfcc_3_mean', 'mfcc_4_mean', 'mfcc_5_mean', 'mfcc_6_mean', 'mfcc_7_mean', 'mfcc_8_mean', 'mfcc_9_mean', 'mfcc_10_mean', 'mfcc_11_mean', 'mfcc_12_mean', 'mfcc_13_mean', 'chroma_1_mean', 'chroma_2_mean', 'chroma_3_mean', 'chroma_4_mean', 'chroma_5_mean', 'chroma_6_mean', 'chroma_7_mean', 'chroma_8_mean', 'chroma_9_mean', 'chroma_10_mean', 'chroma_11_mean', 'chroma_12_mean', 'chroma_std_mean', 'delta zcr_mean', 'delta energy_mean', 'delta energy_entropy_mean', 'delta spectral_centroid_mean', 'delta spectral_spread_mean', 'delta spectral_entropy_mean', 'delta spectral_flux_mean', 'delta spectral_rolloff_mean', 'delta mfcc_1_mean', 'delta mfcc_2_mean', 'delta mfcc_3_mean', 'delta mfcc_4_mean', 'delta mfcc_5_mean', 'delta mfcc_6_mean', 'delta mfcc_7_mean', 'delta mfcc_8_mean', 'de

AttributeError: module 'tensorflow_estimator.python.estimator.api._v2.estimator' has no attribute 'inputs'